## Preprocessing for ENSANUT 2012 - Componente de SALUD 

https://ensanut.insp.mx/encuestas/ensanut2012/descargas.php

- Información sobre el hogar
- Información sobre los integrantes del hogar
- Relación entre folio de hogar y folio de vivenda
- Cuestionario individual: Menores de 0 a 9 años de edad

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os


## Limpieza folio_vivienda


In [2]:
# read csv
folio_vivienda= pd.read_csv("../data/raw_salud/folio_vivienda.csv")

In [3]:
folio_vivienda.columns

Index(['folio_c', 'folio_v'], dtype='object')

Vars: 
- folio_c = folio consecutivo
- folio_v = folio de vivienda

In [4]:
folio_vivienda.isna().sum()

folio_c    0
folio_v    0
dtype: int64

In [5]:
folio_vivienda = folio_vivienda.rename(columns={
    "folio_c": "folio_consecutivo",
    "folio_v": "folio_vivienda"
})

folio_vivienda

,folio_consecutivo,folio_vivienda
0,10321,010321M
1,10322,010321M
2,10800,010800M
3,10801,010800M
4,10842,010842M
...,...,...
50523,321855,321855
50524,321857,321857
50525,321858,321858
50526,321860,321860


In [6]:
folio_vivienda.dtypes

folio_consecutivo     int64
folio_vivienda       object
dtype: object

In [7]:
folio_vivienda["folio_consecutivo"] = folio_vivienda["folio_consecutivo"].astype(str)
folio_vivienda["folio_vivienda"] = folio_vivienda["folio_vivienda"].astype(str)

print(folio_vivienda.dtypes)

folio_consecutivo    object
folio_vivienda       object
dtype: object


---

### Limpieza Hogar_hogar

In [14]:
Hogar_hogar = pd.read_csv(
    "../data/raw_salud/Hogar_hogar.csv",
    encoding="latin1",  # ← este sí lo lee completo sin error
    na_values=["", " ", "NA"],
    low_memory=False
)


In [15]:
Hogar_hogar.columns = hogar.columns.str.replace("ï»¿", "")


In [17]:
# Filtra columnas tipo texto
cols_obj = Hogar_hogar.select_dtypes(include="object").columns

# Buscar valores con caracteres extraños
for col in cols_obj:
    raros = Hogar_hogar[col].dropna().astype(str).str.contains("�", regex=False).sum()
    if raros > 0:
        print(f"{col}: {raros} valores con caracteres extraños")


In [18]:
Hogar_hogar.columns.tolist()

['folio',
 'entidad',
 'munici',
 'locali',
 'deciles',
 'quintiles',
 'h101',
 'h102',
 'h103',
 'h205',
 'h206',
 'h501',
 'h502',
 'h503',
 'h504',
 'h505',
 'h506',
 'h507',
 'h508',
 'h509',
 'h510',
 'h511',
 'h512',
 'h513',
 'h514',
 'h515',
 'h516',
 'h516a',
 'h517',
 'h518',
 'h519',
 'h519esp',
 'h520',
 'h521',
 'h521esp',
 'h522',
 'h522esp',
 'h523',
 'h524a',
 'h524b',
 'h524c',
 'h524d',
 'h524e',
 'h60101',
 'h60102',
 'h60103',
 'h60104',
 'h60105',
 'h60106',
 'h60107',
 'h60108',
 'h60109',
 'h60110',
 'h60111',
 'h60112',
 'h60113',
 'h60114',
 'h60115',
 'h60116',
 'h60117',
 'h60118',
 'h60119',
 'h60120',
 'h60121',
 'h60122',
 'h60123',
 'h60124',
 'h60125',
 'h70101',
 'h70102',
 'h70103',
 'h70104',
 'h70105',
 'h70106',
 'h70107',
 'h70108',
 'h70109',
 'h70110',
 'h70111',
 'h70112',
 'h70113',
 'h70114',
 'h70115',
 'h70116',
 'h70117',
 'h70201',
 'h70202',
 'h70203',
 'h70204',
 'h70205',
 'h70206',
 'h70207',
 'h70301',
 'h70302',
 'h70303',
 'h70304',

In [23]:
columnas_utiles = ['folio', 'entidad', 'munici', 'locali', 'deciles', 'est_urb', 'est_marg', 'pondeh']
hogar_util = Hogar_hogar[columnas_utiles]

hogar_util = hogar_util.rename(columns={
    "folio": "folio_consecutivo"})

Vars:
- folio_consecutivo = identificador unico del hogar
- entidad = clave de la entidad federativa (estado)
- munici = clave del municipio
- locali = clave de la localidad
- deciles = decil de ingreso del hogar
- est_urb = indicador de zona urbana (1 si es urbano, 0 si no)
- est_marg = indicador de marginacion de la zona
- pondeh = ponderador del hogar (factor de expansion muestral)

In [24]:
hogar_util.isna().sum().sort_values(ascending=False)

deciles              2
folio_consecutivo    0
entidad              0
munici               0
locali               0
est_urb              0
est_marg             0
pondeh               0
dtype: int64

In [25]:
hogar_util.shape


(50528, 8)

In [26]:
for col in hogar_util.columns:
    print(f"{col}: {hogar_util[col].nunique()} valores únicos")


folio_consecutivo: 50528 valores únicos
entidad: 32 valores únicos
munici: 177 valores únicos
locali: 323 valores únicos
deciles: 10 valores únicos
est_urb: 3 valores únicos
est_marg: 2 valores únicos
pondeh: 2833 valores únicos


In [28]:
hogar_util['folio_consecutivo'].duplicated().sum()


0

---

### Hogar_integrantes


---

### Menores